In [1]:
import pandas as pd
import os
from pathlib import Path
import pickle

pd.set_option('display.max_columns', None)

Preprocessing Function

In [2]:
def preprocess_station(station_dict):
    nwm_df = station_dict['nwm']
    usgs_df = station_dict['usgs']

    # Bring DateTime back as a column if it's the index
    if usgs_df.index.name == 'DateTime':
        usgs_df = usgs_df.reset_index()

    # Handle datetime conversion and strip timezone
    usgs_df['DateTime'] = pd.to_datetime(usgs_df['DateTime']).dt.round('h')
    usgs_df['DateTime'] = usgs_df['DateTime'].dt.tz_localize(None)

    nwm_df['model_output_valid_time'] = pd.to_datetime(nwm_df['model_output_valid_time'])
    nwm_df['model_output_valid_time'] = nwm_df['model_output_valid_time'].dt.tz_localize(None)

    # Find the streamflow column in NWM data
    flow_col = next((col for col in nwm_df.columns if 'streamflow' in col.lower()), None)
    if flow_col is None:
        raise KeyError("Could not find a streamflow column in NWM data")

    # Merge on timestamp
    merged = pd.merge(
        nwm_df,
        usgs_df,
        how='inner',
        left_on='model_output_valid_time',
        right_on='DateTime'
    )

    merged = merged[[
        'model_initialization_time',
        'model_output_valid_time',
        flow_col,
        'USGSFlowValue'
    ]]

    merged = merged.rename(columns={
        flow_col: 'NWM_streamflow',
        'USGSFlowValue': 'USGS_streamflow'
    })

    return merged

Load Raw Data

In [3]:
pickle_path = Path("../data/processed/station_data.pkl")
with open(pickle_path, 'rb') as f:
    station_data = pickle.load(f)

assert 'station1' in station_data and 'station2' in station_data

Preprocess Both Stations

In [4]:
station1_df = preprocess_station(station_data['station1'])
station2_df = preprocess_station(station_data['station2'])

Add lead_time, month, day

In [5]:
for df in [station1_df, station2_df]:
    df['model_initialization_time'] = pd.to_datetime(df['model_initialization_time'])
    df['lead_time'] = (df['model_output_valid_time'] - df['model_initialization_time']).dt.total_seconds() / 3600
    df['year'] = df['model_output_valid_time'].dt.year
    df['month'] = df['model_output_valid_time'].dt.month
    df['day'] = df['model_output_valid_time'].dt.day

Save Final Output

In [6]:
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

station1_df.to_parquet(output_dir / "station1_processed.parquet", index=False)
station2_df.to_parquet(output_dir / "station2_processed.parquet", index=False)

print("Saved station1_processed.parquet and station2_processed.parquet")

Saved station1_processed.parquet and station2_processed.parquet
